# Machine Learning Engineer Nanodegree
## Supervised Learning
## Project 2: Building a Student Intervention System

Welcome to the second project of the Machine Learning Engineer Nanodegree! In this notebook, some template code has already been provided for you, and it will be your job to implement the additional functionality necessary to successfully complete this project. Sections that begin with **'Implementation'** in the header indicate that the following block of code will require additional functionality which you must provide. Instructions will be provided for each section and the specifics of the implementation are marked in the code block with a `'TODO'` statement. Please be sure to read the instructions carefully!

In addition to implementing code, there will be questions that you must answer which relate to the project and your implementation. Each section where you will answer a question is preceded by a **'Question X'** header. Carefully read each question and provide thorough answers in the following text boxes that begin with **'Answer:'**. Your project submission will be evaluated based on your answers to each of the questions and the implementation you provide.  

>**Note:** Code and Markdown cells can be executed using the **Shift + Enter** keyboard shortcut. In addition, Markdown cells can be edited by typically double-clicking the cell to enter edit mode.

### Question 1 - Classification vs. Regression
*Your goal for this project is to identify students who might need early intervention before they fail to graduate. Which type of supervised learning problem is this, classification or regression? Why?*

**Answer: **
This is a classification problem. The reason is that at the end of the day, the model should give us a clear answer whether we are going to exercise an early intervention or not. So, it seems logical to have a categorical output in the form of a label. Even if for some reason we would design a continuous output somehow, we would still need to discretize it in order for the model to recommend a course of action so we would be effectively turning our regression into a classification. 

## Exploring the Data
Run the code cell below to load necessary Python libraries and load the student data. Note that the last column from this dataset, `'passed'`, will be our target label (whether the student graduated or didn't graduate). All other columns are features about each student.

In [57]:
# Import libraries
import numpy as np
import pandas as pd
from time import time
from sklearn.metrics import f1_score
from __future__ import division

# Read student data
student_data = pd.read_csv("student-data.csv")
print "Student data read successfully!"

Student data read successfully!


### Implementation: Data Exploration
Let's begin by investigating the dataset to determine how many students we have information on, and learn about the graduation rate among these students. In the code cell below, you will need to compute the following:
- The total number of students, `n_students`.
- The total number of features for each student, `n_features`.
- The number of those students who passed, `n_passed`.
- The number of those students who failed, `n_failed`.
- The graduation rate of the class, `grad_rate`, in percent (%).


In [58]:
print student_data.loc[student_data['passed'] == 'yes'].shape[0]

265


In [59]:

# TODO: Calculate number of students
n_students = student_data.shape[0] #the first dimension gives the number of rows 

# TODO: Calculate number of features
n_features = student_data.shape[1] - 1 #the last column is the output label and thus not a feature

# TODO: Calculate passing students
n_passed = student_data.loc[student_data['passed'] == 'yes'].shape[0]

# TODO: Calculate failing students
n_failed = n_students - n_passed 

# TODO: Calculate graduation rate
grad_rate = (n_passed / n_students) * 100

# Print the results
print "Total number of students: {}".format(n_students)
print "Number of features: {}".format(n_features)
print "Number of students who passed: {}".format(n_passed)
print "Number of students who failed: {}".format(n_failed)
print "Graduation rate of the class: {:.2f}%".format(grad_rate)

Total number of students: 395
Number of features: 30
Number of students who passed: 265
Number of students who failed: 130
Graduation rate of the class: 67.09%


## Preparing the Data
In this section, we will prepare the data for modeling, training and testing.

### Identify feature and target columns
It is often the case that the data you obtain contains non-numeric features. This can be a problem, as most machine learning algorithms expect numeric data to perform computations with.

Run the code cell below to separate the student data into feature and target columns to see if any features are non-numeric.

In [60]:
# Extract feature columns
feature_cols = list(student_data.columns[:-1])

# Extract target column 'passed'
target_col = student_data.columns[-1] 

# Show the list of columns
print "Feature columns:\n{}".format(feature_cols)
print "\nTarget column: {}".format(target_col)

# Separate the data into feature data and target data (X_all and y_all, respectively)
X_all = student_data[feature_cols]
y_all = student_data[target_col]

# Show the feature information by printing the first five rows
print "\nFeature values:"
print X_all.head()

Feature columns:
['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu', 'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']

Target column: passed

Feature values:
  school sex  age address famsize Pstatus  Medu  Fedu     Mjob      Fjob  \
0     GP   F   18       U     GT3       A     4     4  at_home   teacher   
1     GP   F   17       U     GT3       T     1     1  at_home     other   
2     GP   F   15       U     LE3       T     1     1  at_home     other   
3     GP   F   15       U     GT3       T     4     2   health  services   
4     GP   F   16       U     GT3       T     3     3    other     other   

    ...    higher internet  romantic  famrel  freetime goout Dalc Walc health  \
0   ...       yes       no        no       4         3     4    1    1      3   
1   ...       

### Preprocess Feature Columns

As you can see, there are several non-numeric columns that need to be converted! Many of them are simply `yes`/`no`, e.g. `internet`. These can be reasonably converted into `1`/`0` (binary) values.

Other columns, like `Mjob` and `Fjob`, have more than two values, and are known as _categorical variables_. The recommended way to handle such a column is to create as many columns as possible values (e.g. `Fjob_teacher`, `Fjob_other`, `Fjob_services`, etc.), and assign a `1` to one of them and `0` to all others.

These generated columns are sometimes called _dummy variables_, and we will use the [`pandas.get_dummies()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html?highlight=get_dummies#pandas.get_dummies) function to perform this transformation. Run the code cell below to perform the preprocessing routine discussed in this section.

In [61]:
def preprocess_features(X):
    ''' Preprocesses the student data and converts non-numeric binary variables into
        binary (0/1) variables. Converts categorical variables into dummy variables. '''
    
    # Initialize new output DataFrame
    output = pd.DataFrame(index = X.index)

    # Investigate each feature column for the data
    for col, col_data in X.iteritems():
        
        # If data type is non-numeric, replace all yes/no values with 1/0
        if col_data.dtype == object:
            col_data = col_data.replace(['yes', 'no'], [1, 0])

        # If data type is categorical, convert to dummy variables
        if col_data.dtype == object:
            # Example: 'school' => 'school_GP' and 'school_MS'
            col_data = pd.get_dummies(col_data, prefix = col)  
        
        # Collect the revised columns
        output = output.join(col_data)
    
    return output

X_all = preprocess_features(X_all)
print "Processed feature columns ({} total features):\n{}".format(len(X_all.columns), list(X_all.columns))

Processed feature columns (48 total features):
['school_GP', 'school_MS', 'sex_F', 'sex_M', 'age', 'address_R', 'address_U', 'famsize_GT3', 'famsize_LE3', 'Pstatus_A', 'Pstatus_T', 'Medu', 'Fedu', 'Mjob_at_home', 'Mjob_health', 'Mjob_other', 'Mjob_services', 'Mjob_teacher', 'Fjob_at_home', 'Fjob_health', 'Fjob_other', 'Fjob_services', 'Fjob_teacher', 'reason_course', 'reason_home', 'reason_other', 'reason_reputation', 'guardian_father', 'guardian_mother', 'guardian_other', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']


### Implementation: Training and Testing Data Split
So far, we have converted all _categorical_ features into numeric values. For the next step, we split the data (both features and corresponding labels) into training and test sets. In the following code cell below, you will need to implement the following:
- Randomly shuffle and split the data (`X_all`, `y_all`) into training and testing subsets.
  - Use 300 training points (approximately 75%) and 95 testing points (approximately 25%).
  - Set a `random_state` for the function(s) you use, if provided.
  - Store the results in `X_train`, `X_test`, `y_train`, and `y_test`.

In [62]:
# TODO: Import any additional functionality you may need here
#from sklearn.cross_validation import train_test_split
from sklearn.cross_validation import StratifiedShuffleSplit
# TODO: Set the number of training points
num_train = 300 #as per the instructions above

# Set the number of testing points
num_test = X_all.shape[0] - num_train

# TODO: Shuffle and split the dataset into the number of training and testing points above
sss = StratifiedShuffleSplit(y_all, n_iter = 1, test_size = num_test, train_size = num_train, random_state = 42)
for train_index, test_index in sss:
    X_train, X_test = X_all.loc[train_index], X_all.loc[test_index]
    y_train, y_test = y_all.loc[train_index], y_all.loc[test_index]
#X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, test_size = num_test, random_state = 42)

# Show the results of the split
print "Training set has {} samples.".format(X_train.shape[0])
print "Testing set has {} samples.".format(X_test.shape[0])

Training set has 300 samples.
Testing set has 95 samples.


## Training and Evaluating Models
In this section, you will choose 3 supervised learning models that are appropriate for this problem and available in `scikit-learn`. You will first discuss the reasoning behind choosing these three models by considering what you know about the data and each model's strengths and weaknesses. You will then fit the model to varying sizes of training data (100 data points, 200 data points, and 300 data points) and measure the F<sub>1</sub> score. You will need to produce three tables (one for each model) that shows the training set size, training time, prediction time, F<sub>1</sub> score on the training set, and F<sub>1</sub> score on the testing set.

### Question 2 - Model Application
*List three supervised learning models that are appropriate for this problem. What are the general applications of each model? What are their strengths and weaknesses? Given what you know about the data, why did you choose these models to be applied?*

**Answer: **
We are asked to model the factors that predict how likely a student is to pass their high school final exam using an efficient model that uses the least amount of computation costs.

Therefore, we need to find a balance between model expressivity and complexity. After looking at the correlation matrix there are some variables that are correlated (like Dalc and Walc with a correlation coefficient of 0.64) but many have low correlation coefficient (0.2 and below) so in order to keep things simple and of low complexity I thought that possible models (having in mind we have 48 features and 395 data vectors/observations/students) can be: SVM, NB (due to the low correlation which stands for columns “relatively” independent) and KNN which although it a lazy algorithm it has low complexity of lgn+k for query and constant for learning (being lazy….) and the space is linear because we need to hold the data which seems reasonable under the project description.

I thought about decision trees but with 48 features the tree can over fit rather quickly so if these ones won't work we can try to tune the tree depth and explore this as well, but for this number of features it probably won’t be a good idea as one of the initial models.

It is important to note that domain knowledge, which I lack :-), will come handy here. For example, KNN assumes that all features matter equally which I don’t think is true but it is a good starting point assumption when you don’t really know the domain. 

The SVM main weakness is the fact it is a binary classifier (which lands perfectly to our problem but IN GENERAL is a weakness), and therefore for multi-class classification only pair-wise classifications can be used. Its strengths are that it is great for linearly separable data but also for non-linear separable data due to the possible different kernel functions (where domain knowledge again comes into play) which will give high accuracy in the context of high dimension (we have 48 features). Also, SVM is more robust. The algorithm spends more time in training but achieves a better F1 score for prediction and thus more accurate.

For NB, its main weakness is the assumption that all features must be conditionally independent from each other which does not seem that bad having in mind the low correlation coefficients between most of the features and since its main strength is its simplicity and efficiency for training in terms of the total number of computations needed, I thought we should give it a go.

For KNN, the main strength is the integration of additional training data (which for a data set like this will happen a lot) and the main assumptions are locality, that is, data points that are close to one another in distance are expected to have similar value which seems very reasonable to me for these data set, while the main weakness being that as the number of features grow, the amount of data required for accurate generalization grows exponentially, but I think it worth a shot. 
Also, another weakness of KNN is that because it is an instance based algorithm it is heavily depedant upon the data we feed into it and thus on the train/test split. The fact that we used StratifiedShuffleSplit should help a bit but it is a general weakness.

As a general and last remark, if some of these won’t work I will try to choose other models later on (NN, for example) later on but as a start these ones seem promising.  



### Setup
Run the code cell below to initialize three helper functions which you can use for training and testing the three supervised learning models you've chosen above. The functions are as follows:
- `train_classifier` - takes as input a classifier and training data and fits the classifier to the data.
- `predict_labels` - takes as input a fit classifier, features, and a target labeling and makes predictions using the F<sub>1</sub> score.
- `train_predict` - takes as input a classifier, and the training and testing data, and performs `train_clasifier` and `predict_labels`.
 - This function will report the F<sub>1</sub> score for both the training and testing data separately.

In [63]:
def train_classifier(clf, X_train, y_train):
    ''' Fits a classifier to the training data. '''
    
    # Start the clock, train the classifier, then stop the clock
    start = time()
    clf.fit(X_train, y_train)
    end = time()
    
    # Print the results
    print "Trained model in {:.4f} seconds".format(end - start)

    
def predict_labels(clf, features, target):
    ''' Makes predictions using a fit classifier based on F1 score. '''
    
    # Start the clock, make predictions, then stop the clock
    start = time()
    y_pred = clf.predict(features)
    end = time()
    
    # Print and return results
    print "Made predictions in {:.4f} seconds.".format(end - start)
    return f1_score(target.values, y_pred, pos_label='yes')


def train_predict(clf, X_train, y_train, X_test, y_test):
    ''' Train and predict using a classifer based on F1 score. '''
    
    # Indicate the classifier and the training set size
    print "Training a {} using a training set size of {}. . .".format(clf.__class__.__name__, len(X_train))
    
    # Train the classifier
    train_classifier(clf, X_train, y_train)
    
    # Print the results of prediction for both training and testing
    print "F1 score for training set: {:.4f}.".format(predict_labels(clf, X_train, y_train))
    print "F1 score for test set: {:.4f}.".format(predict_labels(clf, X_test, y_test))

### Implementation: Model Performance Metrics
With the predefined functions above, you will now import the three supervised learning models of your choice and run the `train_predict` function for each one. Remember that you will need to train and predict on each classifier for three different training set sizes: 100, 200, and 300. Hence, you should expect to have 9 different outputs below — 3 for each model using the varying training set sizes. In the following code cell, you will need to implement the following:
- Import the three supervised learning models you've discussed in the previous section.
- Initialize the three models and store them in `clf_A`, `clf_B`, and `clf_C`.
 - Use a `random_state` for each model you use, if provided.
 - **Note:** Use the default settings for each model — you will tune one specific model in a later section.
- Create the different training set sizes to be used to train each model.
 - *Do not reshuffle and resplit the data! The new training points should be drawn from `X_train` and `y_train`.*
- Fit each model with each training set size and make predictions on the test set (9 in total).  
**Note:** Three tables are provided after the following code cell which can be used to store your results.

In [64]:
# TODO: Import the three supervised learning models from sklearn
from sklearn import svm
from sklearn import naive_bayes
from sklearn import neighbors
seed = 42
# TODO: Initialize the three models
clf_A = svm.SVC(random_state = seed)
clf_B = naive_bayes.GaussianNB() #no need for random_state according with documentation
clf_C = neighbors.KNeighborsClassifier() #no need for random_state according with documentation

# TODO: Set up the training set sizes
trainSizes = [100,200,300]
#X_train_100 = X_train[0:100]
#y_train_100 = y_train[0:100]

#X_train_200 = X_train[0:200]
#y_train_200 = y_train[0:200]

#X_train_300 = X_train[0:300]
#y_train_300 = y_train[0:300]

# TODO: Execute the 'train_predict' function for each classifier and each training set size
models = [clf_A, clf_B, clf_C]
for clf in models:
    print "\n{}: \n".format(clf.__class__.__name__)
    for n in trainSizes:
        train_predict(clf, X_train[:n], y_train[:n], X_test, y_test)
#trainingSetSizes = {1:(X_train_100, y_train_100), 2:(X_train_200, y_train_200), 3:(X_train_300, y_train_300)}
#for key in trainingSetSizes:
#    trainTuple = trainingSetSizes[key]
#    train_predict(clf_A, trainTuple[0], trainTuple[1], X_test, y_test)
#    train_predict(clf_B, trainTuple[0], trainTuple[1], X_test, y_test)
#    train_predict(clf_C, trainTuple[0], trainTuple[1], X_test, y_test)
# train_predict(clf, X_train, y_train, X_test, y_test)


SVC: 

Training a SVC using a training set size of 100. . .
Trained model in 0.0060 seconds
Made predictions in 0.0040 seconds.
F1 score for training set: 0.8354.
Made predictions in 0.0030 seconds.
F1 score for test set: 0.8025.
Training a SVC using a training set size of 200. . .
Trained model in 0.0130 seconds
Made predictions in 0.0100 seconds.
F1 score for training set: 0.8431.
Made predictions in 0.0050 seconds.
F1 score for test set: 0.8105.
Training a SVC using a training set size of 300. . .
Trained model in 0.0240 seconds
Made predictions in 0.0180 seconds.
F1 score for training set: 0.8664.
Made predictions in 0.0060 seconds.
F1 score for test set: 0.8052.

GaussianNB: 

Training a GaussianNB using a training set size of 100. . .
Trained model in 0.0020 seconds
Made predictions in 0.0010 seconds.
F1 score for training set: 0.7752.
Made predictions in 0.0010 seconds.
F1 score for test set: 0.6457.
Training a GaussianNB using a training set size of 200. . .
Trained model in 0

### Tabular Results
Edit the cell below to see how a table can be designed in [Markdown](https://github.com/adam-p/markdown-here/wiki/Markdown-Cheatsheet#tables). You can record your results from above in the tables provided.

** Classifer 1 - svm.SVC**  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------:|
| 100               |        0.006            |    0.003               |    0.8354        |    0.8025      |
| 200               |        0.0130           |    0.005               |    0.8431        |    0.8105      |
| 300               |        0.0240           |    0.006               |    0.8664        |    0.8052      |

** Classifer 2 - naive_bayes.GaussianNB **  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |     0.002               |  0.001                 | 0.7752           |     0.6457      |
| 200               |     0.003               |  0.001                 | 0.8060           |     0.7218      |
| 300               |     0.002               |  0.001                 | 0.8134           |     0.7761      |

** Classifer 3 - neighbors.KNeighborsClassifier**  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |  0.002                  | 0.004                  | 0.8252           | 0.7586          |
| 200               |  0.002                  | 0.006                  | 0.8097           | 0.7857          |
| 300               |  0.002                  | 0.010                  | 0.8539           | 0.8138          |

## Choosing the Best Model
In this final section, you will choose from the three supervised learning models the *best* model to use on the student data. You will then perform a grid search optimization for the model over the entire training set (`X_train` and `y_train`) by tuning at least one parameter to improve upon the untuned model's F<sub>1</sub> score. 

### Question 3 - Chosing the Best Model
*Based on the experiments you performed earlier, in one to two paragraphs, explain to the board of supervisors what single model you chose as the best model. Which model is generally the most appropriate based on the available data, limited resources, cost, and performance?*

**Answer: **
Based on the results of training and testing for different data set sizes, I was considering either SVM or KNN and decided that the single model we should use is SVM.
My selection criteria are the model accuracy and the training and prediction cost. Since resources are scarce we need a model whose training time and prediction time are relatively low. In this regard KNN is the worst (as can be expected but at 300 data points is worth by a factor of 10 than NB and 1.6 more than SVM). The second issue is related to the learning curves. The learning curve for NB is  not as good as SVM and KNN in terms of accuracy, and for SVM and KNN it stabilizes at relatively similar values and gap between the F1 training score and F1 test score and therefore the differentiator between the two was the prediction time and training time for which the SVM seems to be better.
SVM is generally a great model for binary classification which is basically our current task and so it lands naturally to our problem. Maybe we need a non-linear dividing surface in which case we can try to tune the model using different kernel functions to see how it performs. 
Also, the factor of 1.6 in prediction time in favor of SVM "wins" the tradeoff of accuracy with the full training size set (300) in which KNN has an F1 score that represents a 1.05% improvement with respect to SVM (0.8138 for KNN versus 0.8052 of SVM) and if we consider that somewhere between 200 and 300 training points lies the balance point in the learning curve it is even less. 
At the end of the day, it is a close call, but since KNN suffers from the curse of dimensionality and also being an instance based algorithm it is very sensitive to the actual split of the data to train/test groups, I felt SVM is the more robust choice.
 

### Question 4 - Model in Layman's Terms
*In one to two paragraphs, explain to the board of directors in layman's terms how the final model chosen is supposed to work. For example if you've chosen to use a decision tree or a support vector machine, how does the model go about making a prediction?*

**Answer: **
SVM (support vector machines) is a name for an algorithm in machine learning that is especially effective in what is called binary classification problems. This is a fancy name for a problem like ours in which we need to have a prediction from the model whether a student will pass or fail the final exam which is a binary result since we have only two options.
The way this algorithm works is as follows: we feed into our data set which includes many students with each one having all the variables of interest that we think will have some saying in predicting whether the student will pass or fail the final exam (these are the features, like traveltime, age and famsize. We have 48 in total) and the result for each of these students (pass or fail the exam).
Now, our algorithm will train itself using these data set by trying and to find some dividing “line” that splits the students between the two differently classified groups (pass the exam or not pass the exam). The reason I called it a dividing “line” is that we have more than 2 features so we are trying to find a “line” that separates them which is called an hyperplane in higher dimensions (we have 48). 
The algorithm can and will find many “lines” and the best one is determined by the distance from the closest point of either side, which is called a margin. Our algorithm also have the capability of looking for dividing surfaces which are not linear and thus capture more complicated relations between the features and the outcome. 
Once we have this “line” we have our classifier and when we feed a new student with all the values for all the variables of interest (features) and we will get the prediction that the student will pass or not pass the exam based on where the new student lands on either side of the “line” we have found that separates student who pass from students who don’t pass the final exam.
In order to illustrate these concepts let us look at some visualizations:







In the following figure we have a visualization of a “best” line and as we can see the line seperates student who pass the exam from student s who do not pass the exam and the best line is the one which has the largest distance from the closest student on either side:
![Concept of seperating line and margin ](http://scikit-learn.org/stable/_images/plot_separating_hyperplane_001.png)

Here we can see examples of dividing surfaces and the kernel trick concept. On the right we have a hyperplane and let us assume the red dots represent a student who pass and the blue triangle a student who don’t pass. This is a “line” in more than 2 dimensions, so a new student that the model allocated above the hyperplane will be predicted to pass, for example. The left figure shows the exact same thing in less dimensions and so a non-linear separable surface (a circle in this case) seperates the two student classes. A new student that the model allocate inside the circle will be predicted to fail the final exam. This non-linear separable becomes separable in higher dimensions (the right figure again). The important takeaway is that we can deal with complex relationships and still get a very robust model to help us in making a good prediction:
![Image of kernel trick](http://www.eric-kim.net/eric-kim-net/posts/1/imgs/data_2d_to_3d.png)

### Implementation: Model Tuning
Fine tune the chosen model. Use grid search (`GridSearchCV`) with at least one important parameter tuned with at least 3 different values. You will need to use the entire training set for this. In the code cell below, you will need to implement the following:
- Import [`sklearn.grid_search.gridSearchCV`](http://scikit-learn.org/stable/modules/generated/sklearn.grid_search.GridSearchCV.html) and [`sklearn.metrics.make_scorer`](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.make_scorer.html).
- Create a dictionary of parameters you wish to tune for the chosen model.
 - Example: `parameters = {'parameter' : [list of values]}`.
- Initialize the classifier you've chosen and store it in `clf`.
- Create the F<sub>1</sub> scoring function using `make_scorer` and store it in `f1_scorer`.
 - Set the `pos_label` parameter to the correct value!
- Perform grid search on the classifier `clf` using `f1_scorer` as the scoring method, and store it in `grid_obj`.
- Fit the grid search object to the training data (`X_train`, `y_train`), and store it in `grid_obj`.

In [66]:
# TODO: Import 'GridSearchCV' and 'make_scorer'
from sklearn import grid_search
from sklearn import metrics
from sklearn.svm import SVC
# TODO: Create the parameters list you wish to tune


# here I used two parameters. One is the kernel function for the SVM and the other one is the penalty factor that controls the
# tradeoff between training error and model complexity

seed = 42
parameters = {'kernel':('linear', 'rbf', 'sigmoid','poly'), 'C':[1, 10]}
#parameters = {'kernel':('linear','rbf', 'sigmoid','poly')}
#parameters = {'C':[ 100, 1000]}

# TODO: Initialize the classifier
clf = svm.SVC(random_state = 42)

# TODO: Make an f1 scoring function using 'make_scorer' 
f1_scorer = metrics.make_scorer(metrics.f1_score, pos_label = 'yes')

# TODO: Perform grid search on the classifier using the f1_scorer as the scoring method
grid_obj = grid_search.GridSearchCV(clf, parameters, scoring = f1_scorer)

# TODO: Fit the grid search object to the training data and find the optimal parameters
grid_obj = grid_obj.fit(X_train, y_train)

# Get the estimator
clf = grid_obj.best_estimator_

# Report the final F1 score for training and testing after parameter tuning
print "Tuned model has a training F1 score of {:.4f}.".format(predict_labels(clf, X_train, y_train))
print "Tuned model has a testing F1 score of {:.4f}.".format(predict_labels(clf, X_test, y_test))
print grid_obj.best_estimator_

Made predictions in 0.0080 seconds.
Tuned model has a training F1 score of 0.8664.
Made predictions in 0.0030 seconds.
Tuned model has a testing F1 score of 0.8052.
SVC(C=1, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=42, shrinking=True,
  tol=0.001, verbose=False)


### Question 5 - Final F<sub>1</sub> Score
*What is the final model's F<sub>1</sub> score for training and testing? How does that score compare to the untuned model?*

**Answer: **
The final model's F1 score for training is 0.8664 and for testing is 0.8052. It is the same as the untuned model but it is important to note the in the default settings of SVC the kernel function is 'rbf', that is, radial basis function. This is basically a gaussian so it is not a linear function and it seems that the other functions: linear, sigmoid and poly (with the default of degree = 3) are not doing better than it. That's ok. 
Also, running the grid search on the penalty parameter C does not change the best value of C = 1 (I tried forcing it to include only values of 100 and 1000 to actually see that I get a bad result).
I left all these experiments in the code above as comments. The results reported here are for parameters = {'kernel':('linear', 'rbf', 'sigmoid','poly'), 'C':[1, 10]}.




> **Note**: Once you have completed all of the code implementations and successfully answered each question above, you may finalize your work by exporting the iPython Notebook as an HTML document. You can do this by using the menu above and navigating to  
**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission.